In [1]:
import os
import sagemaker
print(sagemaker.__version__)

from sagemaker.pytorch import PyTorch
from sagemaker.pytorch.model import PyTorchModel
import time
sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/st'

role = sagemaker.get_execution_role()

2.108.0


In [7]:
!rm -rf keybert-pytorch.tar.gz
# !cp pretrained_model/mae_pretrain_vit_base.pth model/checkpoint-best.pth
!cd model && tar -czvf ../keybert-pytorch.tar.gz *

!aws s3 cp keybert-pytorch.tar.gz s3://lhr-data/keybert-pytorch.tar.gz

code/
code/.ipynb_checkpoints/
code/.ipynb_checkpoints/requirements-checkpoint.txt
code/.ipynb_checkpoints/inference-checkpoint.py
code/requirements.txt
code/inference-cn.py
code/inference.py
upload: ./keybert-pytorch.tar.gz to s3://lhr-data/keybert-pytorch.tar.gz


In [8]:
# instance_type = 'local'
instance_type = 'ml.g4dn.xlarge'

# predictor = estimator.deploy(initial_instance_count=1, instance_type=instance_type)
endpoint_name="keybert-" + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
pytorch_model = PyTorchModel(model_data='s3://lhr-data/keybert-pytorch.tar.gz', role=role,
                             entry_point='inference.py', framework_version='1.7.1', py_version='py36', model_server_workers=1)  # TODO set model_server_workers=1 to avoid torchhub bug
predictor = pytorch_model.deploy(instance_type=instance_type, initial_instance_count=1, endpoint_name=endpoint_name)

-----------!

In [9]:
import sagemaker
import json
sm = sagemaker.Session().sagemaker_runtime_client

prompt = "The best part of Amazon SageMaker is that it makes machine learning easy."

response = sm.invoke_endpoint(
    EndpointName=endpoint_name, Body=prompt.encode(encoding="UTF-8"), ContentType="text/csv"
)

result = response["Body"].read().decode()

# result = list(map(float, result[1:-1].split()))
eval(result)

[{'corpus_sentence': 'sagemaker makes machine learning', 'score': 0.8299},
 {'corpus_sentence': 'amazon sagemaker', 'score': 0.7948},
 {'corpus_sentence': 'best amazon sagemaker', 'score': 0.7454},
 {'corpus_sentence': 'best amazon sagemaker makes', 'score': 0.6746},
 {'corpus_sentence': 'amazon sagemaker makes', 'score': 0.6633},
 {'corpus_sentence': 'amazon sagemaker makes machine', 'score': 0.6491},
 {'corpus_sentence': 'sagemaker makes', 'score': 0.6116},
 {'corpus_sentence': 'sagemaker makes machine', 'score': 0.606},
 {'corpus_sentence': 'makes machine learning easy', 'score': 0.5396},
 {'corpus_sentence': 'machine learning easy', 'score': 0.5043}]

In [ ]:
predictor.delete_endpoint()